Bare minimum for 
- consumption of model.joblib
- export of properly formatted .json

# imports

In [1]:
import streamlit as st
import requests
import numpy as np
import pandas as pd

In [23]:
# needs to run an api from AnimeList_API repo on github
url_api ='http://0.0.0.0:8000/predict'

# get and show data from api

## get data 

In [5]:
anime_input = 'Naruto'
predict_size_input = 10
model_input = 'notation'
params = {
    'anime' : anime_input,
    'length' : predict_size_input,
    'model' : model_input
}

response = requests.get(url_api, params=params).json()
prediction = response['prediction']

In [6]:
prediction

{'Naruto: Shippuuden': 0.16479568859309957,
 'Death Note': 0.28978191191199754,
 'Bleach': 0.31862997487679345,
 'Shingeki no Kyojin': 0.3200006084059118,
 'Fullmetal Alchemist: Brotherhood': 0.34244184214998474,
 'Sword Art Online': 0.3471770420949094,
 'Code Geass: Hangyaku no Lelouch': 0.36965757726627224,
 'One Punch Man': 0.37465993577686285,
 'Tokyo Ghoul': 0.3842327761099542,
 'Fairy Tail': 0.38772256888731504}

In [10]:
type(prediction)

dict

## convert dict from json to list

In [17]:
def convert_dict_to_list(dict):
    temp = []
    dictList = []
    for key, value in dict.items():
        temp = [key,value]
        dictList.append(temp)
    return dictList

In [18]:
prediction_convert = convert_dict_to_list(prediction)

In [21]:
prediction_convert

[['Naruto: Shippuuden', 0.16479568859309957],
 ['Death Note', 0.28978191191199754],
 ['Bleach', 0.31862997487679345],
 ['Shingeki no Kyojin', 0.3200006084059118],
 ['Fullmetal Alchemist: Brotherhood', 0.34244184214998474],
 ['Sword Art Online', 0.3471770420949094],
 ['Code Geass: Hangyaku no Lelouch', 0.36965757726627224],
 ['One Punch Man', 0.37465993577686285],
 ['Tokyo Ghoul', 0.3842327761099542],
 ['Fairy Tail', 0.38772256888731504]]

## print list properly

In [28]:
prediction_convert[0][0]

'Naruto: Shippuuden'

In [31]:
prediction_convert[1][0]

'Death Note'

In [34]:
for x in range(len(prediction_convert)):
    print prediction_convert[x],

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(prediction_convert[x], end=" ")? (859047408.py, line 2)

In [37]:
print(' '.join(map(str, prediction_convert))) 

['Naruto: Shippuuden', 0.16479568859309957] ['Death Note', 0.28978191191199754] ['Bleach', 0.31862997487679345] ['Shingeki no Kyojin', 0.3200006084059118] ['Fullmetal Alchemist: Brotherhood', 0.34244184214998474] ['Sword Art Online', 0.3471770420949094] ['Code Geass: Hangyaku no Lelouch', 0.36965757726627224] ['One Punch Man', 0.37465993577686285] ['Tokyo Ghoul', 0.3842327761099542] ['Fairy Tail', 0.38772256888731504]


In [40]:
print(prediction.index.tolist())

AttributeError: 'dict' object has no attribute 'index'

In [62]:
print(prediction_convert[i] for i in range(0, prediction_convert))

TypeError: 'list' object cannot be interpreted as an integer

In [71]:
for i in range(len(prediction_convert)):
    print(prediction_convert[i][0])

Naruto: Shippuuden
Death Note
Bleach
Shingeki no Kyojin
Fullmetal Alchemist: Brotherhood
Sword Art Online
Code Geass: Hangyaku no Lelouch
One Punch Man
Tokyo Ghoul
Fairy Tail


In [75]:
[prediction_convert[i][0] for i in range(len(prediction_convert))]

['Naruto: Shippuuden',
 'Death Note',
 'Bleach',
 'Shingeki no Kyojin',
 'Fullmetal Alchemist: Brotherhood',
 'Sword Art Online',
 'Code Geass: Hangyaku no Lelouch',
 'One Punch Man',
 'Tokyo Ghoul',
 'Fairy Tail']

In [52]:
print([i**2 for i in range(10)])

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
